In [1]:
# Importing Libraries
import import_ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload

import NI_M1
import NI_M2
reload(NI_M1)
reload(NI_M2)


importing Jupyter notebook from NI_M1.ipynb
importing Jupyter notebook from NI_M2.ipynb
importing Jupyter notebook from NI_M1.ipynb
importing Jupyter notebook from NI_M2.ipynb


<module 'NI_M2' from 'NI_M2.ipynb'>

In [4]:

# Import All Episodes dataset
ReadExcel = True

if ReadExcel:
    df_AllEpisodes = pd.read_excel(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\Data\0_Raw\MarchData\All episodes 29-06-21.xlsx')


In [162]:

# 
ColumnList = ['P_Spell_ID', 'diagnosis_group']
df_AllEpisodes_Sub = df_AllEpisodes[ColumnList]

def ICD10_AllEpisodes(df):
    ICD10_All = df.groupby(by='P_Spell_ID')['diagnosis_group'].agg('~'.join)
    df_ICD10All = pd.DataFrame(data={'P_Spell_ID':ICD10_All.index, 'ICD10_All':ICD10_All.values})
    df_ICD10All['ICD10_All'] = df_ICD10All['ICD10_All'].str.split('~')
    df_ICD10All['ICD10_All'] = df_ICD10All['ICD10_All'].apply(set).apply(list)
    
    # Remove -1 from ICD-10 code chain
    for index in df_ICD10All.index:
        if '-1' in df_ICD10All.loc[index, 'ICD10_All']:
            df_ICD10All.loc[index, 'ICD10_All'].remove('-1')
                
    df_ICD10All['ICD10_All'] = df_ICD10All['ICD10_All'].apply('-'.join)
    
    return df_ICD10All

df_ICD10All = ICD10_AllEpisodes(df_AllEpisodes_Sub)
print('Shape of dataset:', df_ICD10All.shape)
df_ICD10All.head(2)


Shape of dataset: (486304, 2)


,P_Spell_ID,ICD10_All
0,1656,J189-J128-B972-J9690-U072
1,2864,G710-J128-B972-L984-I489-U071-F059-R268-J9690-...


## Comparing episodes and final code chain

In [163]:

SpellID = 4804
Filter = df_ICD10All['P_Spell_ID']==SpellID
print('ICD-10 codes across ALL episodes within spell', SpellID, ':')
print(list(df_ICD10All[Filter]['ICD10_All'].values))

print('----------')
Filter = df_AllEpisodes['P_Spell_ID']==SpellID
print('ICD-10 codes for ALL episodes within spell', SpellID, ':')
List = list(set('~'.join(list(df_AllEpisodes[Filter]['diagnosis_group'].values)).split('~')))
List.remove('-1')
df_AllEpisodes[Filter]['diagnosis_group'].values


ICD-10 codes across ALL episodes within spell 4804 :
['Z950-A415-I10X-J128-B964-C799-C61X-H409-N390-J90X-U072-M199']
----------
ICD-10 codes for ALL episodes within spell 4804 :


array(['A415~N390~B964~U072~J128~J90X~C61X~C799~I10X~H409~M199~Z950~-1~-1~-1~-1~-1~-1~-1~-1',
       'A415~N390~B964~U072~J128~J90X~C61X~C799~I10X~H409~M199~Z950~-1~-1~-1~-1~-1~-1~-1~-1',
       'A415~N390~U072~J128~J90X~C61X~C799~I10X~H409~M199~Z950~-1~-1~-1~-1~-1~-1~-1~-1~-1',
       'A415~U072~J128~J90X~C61X~C799~I10X~H409~M199~Z950~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1',
       'A415~U072~J128~C61X~C799~I10X~H409~M199~Z950~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1'],
      dtype=object)

In [164]:

Save = False
if Save:
    df_ICD10All.to_csv(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\ICD10_AllEpisodes.csv')
    